In [1]:
import pandas as pd
import numpy as np
import json
import requests

In [2]:
import sys
sys.path.append('../')

In [3]:
from fangraphsdownload import updatefangraphs

In [4]:
def GetPosition (array):
    if isinstance(array,np.ndarray):
        if array.__contains__('C'):
            return 'C'
        if array.__contains__('2B'):
            return '2B'
        if array.__contains__('3B'):
            return '3B'
        if array.__contains__('SS'):
            return 'SS'
        if array.__contains__('1B'):
            return '1B'
        if array.__contains__('OF'):
            return 'OF'
        else:
            return 'Util'
    else: 
        return 'Util'

In [5]:
atcpit=pd.read_json('https://www.fangraphs.com/api/projections?stats=pit&type=atc')
atcbat=pd.read_json('https://www.fangraphs.com/api/projections?stats=bat&type=atc')

In [6]:
atcbat

,Name,Team,ShortName,G,AB,PA,H,1B,2B,3B,...,InterSK,IntraSD,ADP,Pos,minpos,teamid,League,PlayerName,playerids,.
0,"<a href=""statss.aspx?playerid=15640&position=O...",NYY,Yankees,149,545,645,155,87,25,1,...,-0.559173,1.144230,3.220000,0.142045,OF,9.0,AL,Aaron Judge,15640,
1,"<a href=""statss.aspx?playerid=20123&position=O...",SDP,Padres,152,522,665,147,88,27,2,...,-0.405540,0.822719,10.640000,0.006579,OF,29.0,NL,Juan Soto,20123,
2,"<a href=""statss.aspx?playerid=10155&position=O...",LAA,Angels,139,511,603,136,67,27,3,...,-0.971887,1.386280,20.000000,0.059322,OF,1.0,AL,Mike Trout,10155,
3,"<a href=""statss.aspx?playerid=13510&position=3...",CLE,Guardians,151,570,651,156,85,36,4,...,0.127587,0.539221,3.670000,0.191083,3B,5.0,AL,José Ramírez,13510,
4,"<a href=""statss.aspx?playerid=23697&position=O...",SEA,Mariners,146,569,634,158,96,30,3,...,-2.389360,0.717157,4.060000,0.007634,OF,11.0,AL,Julio Rodríguez,23697,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,"<a href=""statss.aspx?playerid=15983&position=O...",None,None,56,169,195,35,22,8,0,...,-0.929591,1.433320,749.760010,0.533333,OF,NaN,None,Clint Frazier,15983,
634,"<a href=""statss.aspx?playerid=15983&position=O...",None,None,56,169,195,35,22,8,0,...,-0.503299,1.571340,749.760010,0.533333,OF,NaN,None,Clint Frazier,15983,
635,"<a href=""statss.aspx?playerid=1744&position=1B...",DET,Tigers,78,292,320,71,54,10,0,...,1.419740,0.833554,743.010010,1.000000,1B/3B,6.0,AL,Miguel Cabrera,1744,
636,"<a href=""statss.aspx?playerid=21127&position=2...",CIN,Reds,20,50,54,13,10,3,0,...,1.745520,1.795480,999.000000,0.137931,2B,18.0,NL,Alejo Lopez,21127,


In [7]:
hittersmean = atcbat
pitchersmean = atcpit

In [8]:
hittersmean

,Name,Team,ShortName,G,AB,PA,H,1B,2B,3B,...,InterSK,IntraSD,ADP,Pos,minpos,teamid,League,PlayerName,playerids,.
0,"<a href=""statss.aspx?playerid=15640&position=O...",NYY,Yankees,149,545,645,155,87,25,1,...,-0.559173,1.144230,3.220000,0.142045,OF,9.0,AL,Aaron Judge,15640,
1,"<a href=""statss.aspx?playerid=20123&position=O...",SDP,Padres,152,522,665,147,88,27,2,...,-0.405540,0.822719,10.640000,0.006579,OF,29.0,NL,Juan Soto,20123,
2,"<a href=""statss.aspx?playerid=10155&position=O...",LAA,Angels,139,511,603,136,67,27,3,...,-0.971887,1.386280,20.000000,0.059322,OF,1.0,AL,Mike Trout,10155,
3,"<a href=""statss.aspx?playerid=13510&position=3...",CLE,Guardians,151,570,651,156,85,36,4,...,0.127587,0.539221,3.670000,0.191083,3B,5.0,AL,José Ramírez,13510,
4,"<a href=""statss.aspx?playerid=23697&position=O...",SEA,Mariners,146,569,634,158,96,30,3,...,-2.389360,0.717157,4.060000,0.007634,OF,11.0,AL,Julio Rodríguez,23697,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,"<a href=""statss.aspx?playerid=15983&position=O...",None,None,56,169,195,35,22,8,0,...,-0.929591,1.433320,749.760010,0.533333,OF,NaN,None,Clint Frazier,15983,
634,"<a href=""statss.aspx?playerid=15983&position=O...",None,None,56,169,195,35,22,8,0,...,-0.503299,1.571340,749.760010,0.533333,OF,NaN,None,Clint Frazier,15983,
635,"<a href=""statss.aspx?playerid=1744&position=1B...",DET,Tigers,78,292,320,71,54,10,0,...,1.419740,0.833554,743.010010,1.000000,1B/3B,6.0,AL,Miguel Cabrera,1744,
636,"<a href=""statss.aspx?playerid=21127&position=2...",CIN,Reds,20,50,54,13,10,3,0,...,1.745520,1.795480,999.000000,0.137931,2B,18.0,NL,Alejo Lopez,21127,


In [9]:
hittersmean['OBPOnTeam'] = (hittersmean['PA']*hittersmean['OBP']+2037.4)/(hittersmean['PA']+6100)
hittersmean['SLGOnTeam'] = (hittersmean['AB']*hittersmean['SLG']+2420)/(hittersmean['AB']+5500)
hittersmean['OPSValue'] = (hittersmean['OBPOnTeam']+hittersmean['SLGOnTeam']-.774)/.006

hittersmean['AVGValue'] = ((hittersmean['H']+1768)/(hittersmean['AB']+6100)-.267)/.0024
hittersmean['value'] = hittersmean['OPSValue']+(hittersmean['R']/8.92+hittersmean['HR']/7.5+hittersmean['RBI']/8.92+hittersmean['SB']/6.0)

In [10]:
#https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=1&season=2020&month=0&season1=2020&ind=0&team=0&rost=0&age=0&filter=&players=0&startdate=&enddate=
import pybaseball
positions = pybaseball.fielding_stats(2022, qual=0)

In [11]:
positions['Pos']=positions['Pos'].replace(['LF','CF','RF'],'OF')


In [12]:
positions_unique=positions.loc[(positions['G']>=10) | (positions['GS']>=5)].groupby('IDfg')['Pos'].unique()


In [13]:
positions

,IDfg,Season,Name,Team,Pos,G,GS,Inn,PO,A,...,60-90%,# 60-90%,90-100%,# 90-100%,rSZ,rCERA,rTS,FRM,OAA,RAA
0,16725,2022,Jose Trevino,NYY,C,112,89,820.1,828,32,...,NaN,NaN,NaN,NaN,12,3.0,NaN,19.0,NaN,NaN
1,11265,2022,Jonathan Schoop,DET,2B,129,127,1106.0,207,313,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,27.0,20.0
2,18314,2022,Dansby Swanson,ATL,SS,161,161,1433.0,193,391,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,20.0,15.0
3,11739,2022,J.T. Realmuto,PHI,C,133,130,1131.2,1151,49,...,NaN,NaN,NaN,NaN,1,0.0,NaN,1.1,NaN,NaN
4,19352,2022,Sean Murphy,OAK,C,116,116,1004.0,878,40,...,NaN,NaN,NaN,NaN,5,-5.0,NaN,8.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2397,27681,2022,Ken Waldichuk,OAK,P,7,7,34.2,1,2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,-0.3,NaN,NaN
2398,27691,2022,Louie Varland,MIN,P,5,5,26.0,0,2,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0.1,NaN,NaN
2399,27779,2022,Bryce Elder,ATL,P,10,9,54.0,3,4,...,NaN,NaN,NaN,NaN,0,NaN,NaN,-0.1,NaN,NaN
2400,30074,2022,Chase Silseth,LAA,P,7,7,28.2,1,1,...,NaN,NaN,NaN,NaN,0,NaN,NaN,-0.2,NaN,NaN


In [14]:
positions_unique.index=positions_unique.index.map(str)

In [15]:
hitterspos=pd.merge(hittersmean,positions_unique,how='left',left_on='playerids',right_index=True)

In [16]:
hitterspos['Pos']=hitterspos['Pos_y']

In [17]:
hitterspos['BestPos']=hitterspos['Pos'].apply(GetPosition)

hitterspos['rank']=hitterspos.groupby('BestPos')['value'].rank(ascending=False)
hitterspos['utilrank']=hitterspos['value'].rank(ascending=False)

###customized league replacement value
replacementvalue=hitterspos.groupby('BestPos').value.nlargest(10).groupby('BestPos').min().reset_index(name='PosValue')
replacementvalue['UtilValue']=float(hitterspos.loc[hitterspos['utilrank']==80]['value'])

replacementvalue['ReplacementValue']=pd.DataFrame([replacementvalue['UtilValue'], replacementvalue['PosValue']]).min()
#replacementvalue['ReplacementValue']=replacementvalue['UtilValue']
replacementvalue.loc[replacementvalue['BestPos'] == 'Util', 'ReplacementValue'] = replacementvalue['UtilValue']

In [18]:
hittersmean[hittersmean['PlayerName']=='Bryce Harper']

,Name,Team,ShortName,G,AB,PA,H,1B,2B,3B,...,teamid,League,PlayerName,playerids,.,OBPOnTeam,SLGOnTeam,OPSValue,AVGValue,value
115,"<a href=""statss.aspx?playerid=11579&position=O...",PHI,Phillies,75,278,330,78,44,18,1,...,26.0,NL,Bryce Harper,11579,,0.336496,0.443871,1.061254,9.346843,15.40236


In [19]:
replacementvalue

,BestPos,PosValue,UtilValue,ReplacementValue
0,1B,21.260078,20.403903,20.403903
1,2B,20.642349,20.403903,20.403903
2,3B,20.403903,20.403903,20.403903
3,C,15.990753,20.403903,15.990753
4,OF,24.302161,20.403903,20.403903
5,SS,21.212857,20.403903,20.403903
6,Util,11.923120,20.403903,20.403903


In [20]:
hittersfinal=hitterspos.reset_index().merge(replacementvalue,on='BestPos',how='left')

hittersfinal['VORP']=hittersfinal['value']-hittersfinal['ReplacementValue']


In [21]:
hittersfinal

,index,Name,Team,ShortName,G,AB,PA,H,1B,2B,...,value,Pos_y,Pos,BestPos,rank,utilrank,PosValue,UtilValue,ReplacementValue,VORP
0,0,"<a href=""statss.aspx?playerid=15640&position=O...",NYY,Yankees,149,545,645,155,87,25,...,33.674923,[OF],[OF],OF,1.0,1.0,24.302161,20.403903,20.403903,13.271020
1,1,"<a href=""statss.aspx?playerid=20123&position=O...",SDP,Padres,152,522,665,147,88,27,...,29.993282,[OF],[OF],OF,4.0,5.0,24.302161,20.403903,20.403903,9.589379
2,2,"<a href=""statss.aspx?playerid=10155&position=O...",LAA,Angels,139,511,603,136,67,27,...,29.542150,[OF],[OF],OF,5.0,7.0,24.302161,20.403903,20.403903,9.138247
3,3,"<a href=""statss.aspx?playerid=13510&position=3...",CLE,Guardians,151,570,651,156,85,36,...,31.538965,[3B],[3B],3B,1.0,2.0,20.403903,20.403903,20.403903,11.135063
4,4,"<a href=""statss.aspx?playerid=23697&position=O...",SEA,Mariners,146,569,634,158,96,30,...,29.189213,[OF],[OF],OF,6.0,11.0,24.302161,20.403903,20.403903,8.785310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
633,633,"<a href=""statss.aspx?playerid=15983&position=O...",None,None,56,169,195,35,22,8,...,4.931022,NaN,NaN,Util,27.5,386.5,11.923120,20.403903,20.403903,-15.472881
634,634,"<a href=""statss.aspx?playerid=15983&position=O...",None,None,56,169,195,35,22,8,...,4.931022,NaN,NaN,Util,27.5,386.5,11.923120,20.403903,20.403903,-15.472881
635,635,"<a href=""statss.aspx?playerid=1744&position=1B...",DET,Tigers,78,292,320,71,54,10,...,6.959671,NaN,NaN,Util,21.0,333.0,11.923120,20.403903,20.403903,-13.444231
636,636,"<a href=""statss.aspx?playerid=21127&position=2...",CIN,Reds,20,50,54,13,10,3,...,1.207499,[2B],[2B],2B,99.0,537.0,20.642349,20.403903,20.403903,-19.196404


In [22]:

########################
#CHANGE THE COLUMNS BELOW BASED ON SGP VALUES FOR YOUR LEAGUE
pitchersmean['ERA']=((525.55+pitchersmean['ER'])*9/(1350.0+pitchersmean['IP'])-3.5)/-0.1062
pitchersmean['WHIP'] =((1620.0+pitchersmean['H']+pitchersmean['BB'])/(1350.0+pitchersmean['IP'])-1.2)/-0.0201
pitchersmean['value'] = pitchersmean['ERA']+pitchersmean['WHIP']+(pitchersmean['W']/4.05+pitchersmean['K/9']*pitchersmean['IP']/(55.0*9.0)+(pitchersmean['SV']+pitchersmean['HLD'])/10.0)
########################

pitchersmean['utilrank']=pitchersmean['value'].rank(ascending=False)

###customized league replacement value
pitchersmean['ReplacementValue']=float(pitchersmean.loc[pitchersmean['utilrank']==100]['value'])

pitchersmean['VORP']=pitchersmean['value']-pitchersmean['ReplacementValue']


In [23]:
pitchersmean

,Name,Team,ShortName,W,L,ERA,GS,G,SV,HLD,...,ADP,teamid,League,PlayerName,playerids,.,value,utilrank,ReplacementValue,VORP
0,"<a href=""statss.aspx?playerid=19361&position=P...",MIL,Brewers,13,8,0.479975,31,31,0,0,...,14.860000,23.0,NL,Corbin Burnes,19361,,8.875901,2.0,4.265928,4.609972
1,"<a href=""statss.aspx?playerid=19755&position=D...",LAA,Angels,13,8,0.357220,29,29,0,0,...,9.060000,1.0,AL,Shohei Ohtani,19755,,7.946898,4.0,4.265928,3.680970
2,"<a href=""statss.aspx?playerid=16149&position=P...",PHI,Phillies,13,10,-0.060681,31,31,0,0,...,32.750000,26.0,NL,Aaron Nola,16149,,7.925590,6.0,4.265928,3.659661
3,"<a href=""statss.aspx?playerid=10954&position=P...",TEX,Rangers,10,6,0.505884,24,24,0,0,...,33.709999,13.0,AL,Jacob deGrom,10954,,7.528376,9.0,4.265928,3.262447
4,"<a href=""statss.aspx?playerid=27498&position=P...",ATL,Braves,13,7,0.352873,29,29,0,0,...,26.920000,16.0,NL,Spencer Strider,27498,,8.083913,3.0,4.265928,3.817984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,"<a href=""statss.aspx?playerid=10855&position=P...",None,None,1,2,-0.289022,0,32,0,5,...,999.000000,NaN,None,Chasen Shreve,10855,,0.822219,484.0,4.265928,-3.443709
830,"<a href=""statss.aspx?playerid=19800&position=P...",PIT,Pirates,3,5,-0.603951,2,59,2,11,...,735.280029,27.0,NL,Wil Crowe,19800,,1.948480,314.0,4.265928,-2.317448
831,"<a href=""statss.aspx?playerid=9817&position=P""...",None,None,1,1,-0.284269,0,24,0,4,...,999.000000,NaN,None,Sergio Romo,9817,,0.599552,539.0,4.265928,-3.666376
832,"<a href=""statss.aspx?playerid=2608&position=P""...",None,None,2,2,-0.424579,0,32,0,2,...,999.000000,-1.0,None,Jhoulys Chacín,2608,,0.423709,610.0,4.265928,-3.842219


In [24]:
final=pd.concat([hittersfinal,pitchersmean])


In [25]:
final['VORP']=final['VORP'].round(2)

In [26]:
final.sort_values('VORP', ascending=False)

,index,Name,Team,ShortName,G,AB,PA,H,1B,2B,...,K/9,BB/9,K/BB,HR/9,K-BB%,GB%,LOB%,FIP,RA9-WAR,QS
0,0.0,"<a href=""statss.aspx?playerid=15640&position=O...",NYY,Yankees,149,545.0,645.0,155,87.0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,"<a href=""statss.aspx?playerid=13510&position=3...",CLE,Guardians,151,570.0,651.0,156,85.0,36.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,11.0,"<a href=""statss.aspx?playerid=18401&position=O...",ATL,Braves,143,543.0,634.0,147,87.0,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10.0,"<a href=""statss.aspx?playerid=18345&position=O...",HOU,Astros,150,565.0,631.0,156,88.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,"<a href=""statss.aspx?playerid=20123&position=O...",SDP,Padres,152,522.0,665.0,147,88.0,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,588.0,"<a href=""statss.aspx?playerid=17750&position=O...",SDP,Padres,9,21.0,22.0,5,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
525,525.0,"<a href=""statss.aspx?playerid=25353&position=2...",None,None,7,20.0,22.0,4,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,536.0,"<a href=""statss.aspx?playerid=10655&position=C...",TOR,Blue Jays,6,21.0,22.0,5,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
628,628.0,"<a href=""statss.aspx?playerid=22515&position=O...",None,None,7,20.0,21.0,4,3.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
final.sort_values('VORP', ascending=False).nlargest(200,['VORP']).to_csv("ATC.csv")

In [29]:
final[['PlayerName','Team','Pos','BestPos','G','GS','W','SV','value','VORP','ADP','rank','utilrank']].sort_values('VORP', ascending=False).nlargest(200,['VORP']).round(2).to_csv("ATC.txt", sep="\t")